In [1]:
from google.colab import  files
uploaded = files.upload()

Saving best_model.pkl to best_model.pkl


In [2]:
!pip install gradio matplotlib seaborn

import gradio as gr
import joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load model
model = joblib.load('best_model.pkl')

# Assembly line mapping
assembly_line_mapping = {'Shopfloor-L1': 0, 'Shopfloor-L2': 1, 'Shopfloor-L3': 2}

# Prediction function
def predict_downtime(assembly_line, hydraulic_pressure, coolant_pressure, air_system_pressure,
                     coolant_temperature, hydraulic_oil_temperature, spindle_bearing_temperature,
                     spindle_vibration, tool_vibration, spindle_speed, voltage, torque, cutting):

    input_data = pd.DataFrame({
        'Assembly_Line_No': [assembly_line],
        'Hydraulic_Pressure(bar)': [hydraulic_pressure],
        'Coolant_Pressure(bar)': [coolant_pressure],
        'Air_System_Pressure(bar)': [air_system_pressure],
        'Coolant_Temperature': [coolant_temperature],
        'Hydraulic_Oil_Temperature(?C)': [hydraulic_oil_temperature],
        'Spindle_Bearing_Temperature(?C)': [spindle_bearing_temperature],
        'Spindle_Vibration(?m)': [spindle_vibration],
        'Tool_Vibration(?m)': [tool_vibration],
        'Spindle_Speed(RPM)': [spindle_speed],
        'Voltage(volts)': [voltage],
        'Torque(Nm)': [torque],
        'Cutting(kN)': [cutting]
    })

    input_data['Assembly_Line_No'] = input_data['Assembly_Line_No'].map(assembly_line_mapping)

    prediction = model.predict(input_data)[0]
    prediction_mapping = {0: 'Machine_Failure', 1: 'No_Machine_Failure'}
    return prediction_mapping.get(int(round(prediction)), "Unknown")

# Visualization function for input parameters
def show_visualization(assembly_line, hydraulic_pressure, coolant_pressure, air_system_pressure,
                       coolant_temperature, hydraulic_oil_temperature, spindle_bearing_temperature,
                       spindle_vibration, tool_vibration, spindle_speed, voltage, torque, cutting):

    features = {
        'Hydraulic Pressure (bar)': hydraulic_pressure,
        'Coolant Pressure (bar)': coolant_pressure,
        'Air System Pressure (bar)': air_system_pressure,
        'Coolant Temp': coolant_temperature,
        'Hydraulic Oil Temp (°C)': hydraulic_oil_temperature,
        'Spindle Bearing Temp (°C)': spindle_bearing_temperature,
        'Spindle Vibration (?m)': spindle_vibration,
        'Tool Vibration (?m)': tool_vibration,
        'Spindle Speed (RPM)': spindle_speed,
        'Voltage (V)': voltage,
        'Torque (Nm)': torque,
        'Cutting Force (kN)': cutting
    }

    plt.figure(figsize=(10, 6))
    sns.barplot(x=list(features.values()), y=list(features.keys()), palette="viridis")
    plt.title(f'Machine Input Parameters - {assembly_line}')
    plt.xlabel("Value")
    plt.tight_layout()

    return plt.gcf()

# New function to show feature distributions
def show_feature_distributions():
    try:
        if 'df' in globals():
            df_to_plot = df
        else:
            np.random.seed(42)
            data = {
                'Hydraulic_Pressure(bar)': np.random.normal(100, 10, 1000),
                'Coolant_Pressure(bar)': np.random.normal(5, 1, 1000),
                'Air_System_Pressure(bar)': np.random.normal(6, 0.5, 1000),
                'Coolant_Temperature': np.random.normal(25, 5, 1000),
                'Hydraulic_Oil_Temperature(?C)': np.random.normal(40, 5, 1000),
                'Spindle_Bearing_Temperature(?C)': np.random.normal(50, 10, 1000),
                'Spindle_Vibration(?m)': np.random.gamma(2, 2, 1000),
                'Tool_Vibration(?m)': np.random.gamma(1.5, 2, 1000),
                'Spindle_Speed(RPM)': np.random.randint(1000, 5000, 1000),
                'Voltage(volts)': np.random.normal(220, 10, 1000),
                'Torque(Nm)': np.random.normal(50, 15, 1000),
                'Cutting(kN)': np.random.normal(10, 3, 1000)
            }
            df_to_plot = pd.DataFrame(data)

        plt.figure(figsize=(20, 12))

        features = ['Hydraulic_Pressure(bar)', 'Coolant_Pressure(bar)', 'Air_System_Pressure(bar)',
                   'Coolant_Temperature', 'Hydraulic_Oil_Temperature(?C)', 'Spindle_Bearing_Temperature(?C)',
                   'Spindle_Vibration(?m)', 'Tool_Vibration(?m)', 'Spindle_Speed(RPM)', 'Voltage(volts)',
                   'Torque(Nm)', 'Cutting(kN)']

        for i, column in enumerate(features):
            plt.subplot(3, 4, i + 1)
            sns.histplot(df_to_plot[column], kde=True)
            plt.title(f'Distribution of {column}', fontsize=10)
            plt.xticks(rotation=45, ha='right', fontsize=8)
            plt.yticks(fontsize=8)

        plt.tight_layout()
        return plt.gcf()
    except Exception as e:
        print(f"Error in show_feature_distributions: {e}")
        # Return an empty plot if there's an error
        plt.figure(figsize=(10, 6))
        plt.text(0.5, 0.5, "Error showing distributions\nPlease check your data",
                ha='center', va='center')
        return plt.gcf()

# Define Gradio inputs
inputs = [
    gr.Dropdown(["Shopfloor-L1", "Shopfloor-L2", "Shopfloor-L3"], label="Assembly Line"),
    gr.Number(label="Hydraulic Pressure (bar)"),
    gr.Number(label="Coolant Pressure (bar)"),
    gr.Number(label="Air System Pressure (bar)"),
    gr.Number(label="Coolant Temperature"),
    gr.Number(label="Hydraulic Oil Temperature (°C)"),
    gr.Number(label="Spindle Bearing Temperature (°C)"),
    gr.Number(label="Spindle Vibration (?m)"),
    gr.Number(label="Tool Vibration (?m)"),
    gr.Number(label="Spindle Speed (RPM)"),
    gr.Number(label="Voltage (volts)"),
    gr.Number(label="Torque (Nm)"),
    gr.Number(label="Cutting Force (kN)")
]

with gr.Blocks() as demo:
    gr.Markdown("## 🏭 Machine Downtime Prediction")
    gr.Markdown("Enter machine parameters to predict if a failure will occur.")

    with gr.Row():
        with gr.Column():
            for comp in inputs:
                comp.render()

        with gr.Column():
            output_text = gr.Textbox(label="Prediction Result")
            output_plot = gr.Plot(label="Visualization")

            with gr.Row():
                predict_btn = gr.Button("Predict Downtime")
                visualize_btn = gr.Button("Show Input Visualization")
                distributions_btn = gr.Button("Show Feature Distributions")

    input_components = [comp for comp in inputs]
    predict_btn.click(predict_downtime, inputs=input_components, outputs=output_text)
    visualize_btn.click(show_visualization, inputs=input_components, outputs=output_plot)
    distributions_btn.click(show_feature_distributions, inputs=None, outputs=output_plot)

demo.launch()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.1 MB/s eta 0:00:00
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://29706b1f8c90fcd9f8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory 